# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
cities_data_to_load = "C:/Users/paul_/workspace/python-api-challenge/WeatherPy/output_data_weather/cities.csv"
cities_data = pd.read_csv(cities_data_to_load)
cities_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Alugan,12.22,125.48,84.72,80,100,13.18,PH,1603252574
1,Walvis Bay,-22.96,14.51,57.99,82,0,10.29,NaN,1603252574
2,San Policarpo,12.18,125.51,84.67,79,100,14.12,PH,1603252512
3,Hilo,19.73,-155.09,82.40,58,90,6.93,US,1603252297
4,Jamestown,42.10,-79.24,48.99,100,90,5.82,US,1603252574


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [48]:
gmaps.configure(api_key=g_key)
locations = cities_data[["Lat", "Lng"]].astype(float)
humidities = cities_data["Humidity"]
fig = gmaps.figure(center=(20,10), zoom_level=2.4)
heat_layer = gmaps.heatmap_layer(locations, weights=humidities, 
                                 dissipating=False, max_intensity=100,
                                point_radius = 2)

fig.add_layer(heat_layer)
fig
plt.savefig("output_data_vacation/heat_map.png")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
perfect_cities = cities_data.loc[(cities_data['Max Temp']<=85)&(cities_data['Max Temp']>=70)&(cities_data["Wind Speed"]<=10)&(cities_data["Humidity"]<=60)&(cities_data["Cloudiness"]==0)]
perfect_cities.to_csv("output_data_vacation/perfect_cities.csv", index = False, header = True)
perfect_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,Nizwá,22.93,57.53,81.86,11,0,4.23,OM,1603252587
56,Ankazoabo,-22.28,44.52,75.79,40,0,5.77,MG,1603252588
117,Jinji,23.23,110.83,74.64,57,0,9.44,CN,1603252602
167,Sur,22.57,59.53,81.39,50,0,6.76,OM,1603252614
198,Aswān,24.09,32.91,76.84,24,0,8.66,EG,1603252622
224,Araouane,18.90,-3.53,82.17,19,0,8.95,ML,1603252628
332,Bilma,18.69,12.92,74.91,19,0,2.53,NE,1603252654
336,Agadez,19.75,10.25,77.02,18,0,9.71,NE,1603252654
364,Oodweyne,9.41,45.06,73.11,24,0,2.13,SO,1603252661
394,Rāwah,34.48,41.91,71.67,35,0,9.42,IQ,1603252667


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = perfect_cities.copy()
hotel_df["Hotel Name"] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,Nizwá,22.93,57.53,81.86,11,0,4.23,OM,1603252587,
56,Ankazoabo,-22.28,44.52,75.79,40,0,5.77,MG,1603252588,
117,Jinji,23.23,110.83,74.64,57,0,9.44,CN,1603252602,
167,Sur,22.57,59.53,81.39,50,0,6.76,OM,1603252614,
198,Aswān,24.09,32.91,76.84,24,0,8.66,EG,1603252622,
224,Araouane,18.90,-3.53,82.17,19,0,8.95,ML,1603252628,
332,Bilma,18.69,12.92,74.91,19,0,2.53,NE,1603252654,
336,Agadez,19.75,10.25,77.02,18,0,9.71,NE,1603252654,
364,Oodweyne,9.41,45.06,73.11,24,0,2.13,SO,1603252661,
394,Rāwah,34.48,41.91,71.67,35,0,9.42,IQ,1603252667,


In [30]:
params = {
    "radius": 5000,
    "key": g_key,
    "keyword": "lodging",
    "types": "lodging"
}

for index, row in hotel_df.iterrows():
    lat= row["Lat"]
    lng= row["Lng"]
    params["location"]= f'{lat}, {lng}'
    perfect_city = row['City']
    params["vicinity"] = f'{perfect_city}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    results = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results['results'][0]['name']
        print(f"The closest hotel is {results['results'][0]['name']}.")
    except:
        print('Hotel not found')
hotel_df

The closest hotel is Antique Inn - Nizwa.
Hotel not found
Hotel not found
The closest hotel is Al Jumhour Hotel Apartments.
The closest hotel is Kendaka Nubian House.
Hotel not found
The closest hotel is Expeditions Ténére voyages.
Hotel not found
The closest hotel is Eng Hiirey House.
Hotel not found
The closest hotel is Chiyabari Cottage.
The closest hotel is املج 00.
The closest hotel is Swan Hill Resort.
Hotel not found
The closest hotel is Princesse Tsiribihina.
The closest hotel is Odar kanawal.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,Nizwá,22.93,57.53,81.86,11,0,4.23,OM,1603252587,Antique Inn - Nizwa
56,Ankazoabo,-22.28,44.52,75.79,40,0,5.77,MG,1603252588,
117,Jinji,23.23,110.83,74.64,57,0,9.44,CN,1603252602,
167,Sur,22.57,59.53,81.39,50,0,6.76,OM,1603252614,Al Jumhour Hotel Apartments
198,Aswān,24.09,32.91,76.84,24,0,8.66,EG,1603252622,Kendaka Nubian House
224,Araouane,18.90,-3.53,82.17,19,0,8.95,ML,1603252628,
332,Bilma,18.69,12.92,74.91,19,0,2.53,NE,1603252654,Expeditions Ténére voyages
336,Agadez,19.75,10.25,77.02,18,0,9.71,NE,1603252654,
364,Oodweyne,9.41,45.06,73.11,24,0,2.13,SO,1603252661,Eng Hiirey House
394,Rāwah,34.48,41.91,71.67,35,0,9.42,IQ,1603252667,


In [31]:

hotel_df = hotel_df.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,Nizwá,22.93,57.53,81.86,11,0,4.23,OM,1603252587,Antique Inn - Nizwa
56,Ankazoabo,-22.28,44.52,75.79,40,0,5.77,MG,1603252588,
117,Jinji,23.23,110.83,74.64,57,0,9.44,CN,1603252602,
167,Sur,22.57,59.53,81.39,50,0,6.76,OM,1603252614,Al Jumhour Hotel Apartments
198,Aswān,24.09,32.91,76.84,24,0,8.66,EG,1603252622,Kendaka Nubian House
224,Araouane,18.90,-3.53,82.17,19,0,8.95,ML,1603252628,
332,Bilma,18.69,12.92,74.91,19,0,2.53,NE,1603252654,Expeditions Ténére voyages
336,Agadez,19.75,10.25,77.02,18,0,9.71,NE,1603252654,
364,Oodweyne,9.41,45.06,73.11,24,0,2.13,SO,1603252661,Eng Hiirey House
394,Rāwah,34.48,41.91,71.67,35,0,9.42,IQ,1603252667,


In [24]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "location": "22.93, 57.53",
    "keyword": "lodging",
    "radius": 500,
    "type": "hotel",
    "key": g_key,
}
#params["location"]= '19.75, 10.25'
request = requests.get(base_url, params=params).json()

display(request)

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 22.9324205, 'lng': 57.53048740000001},
    'viewport': {'northeast': {'lat': 22.93374867989272,
      'lng': 57.53183602989273},
     'southwest': {'lat': 22.93104902010728, 'lng': 57.52913637010728}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Antique Inn - Nizwa',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 2736,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101113497961416015620">Rock Rock</a>'],
     'photo_reference': 'CmRaAAAA-1rc51H-IU6i9MkMP4LdtdQAzCmkM-h5ThEi5j4xSfEQsYE1BFGWxiABTgxKGSvS0RhtcpG3jAFuLBtcz5fntdOFsfJUk9MqUIMa_xohhtUO6NuXvzii-gaQHeBAMwaAEhDyxJk4dfK0hvhWE0tS225yGhQqDHLIO0oBS6JYyKGlSUxewm6Sqw',
     'width': 3648}],
   'place_id': 'ChIJbdCZdvzxjj4RXBZ2zImIpPs',
   'plus_code': {'compound_code': 'WGJJ+X5 Nizwa, Oman',
    'global_code': '7HJVWGJJ+X5'},
 

In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
vacation_spots = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(vacation_spots)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))